### 3 Types of Recommender Systems: 
        Collaborative Filtering => Recommend based on items bought by User (Same recommendations if two persons buy same product)

        Content-Based Filtering => Focus completely on Content (Same recommendations to two persons if both give same rating to an article)

        Hybrid Filtering (Combining Collaborative and Content-Based)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv(r'F:\Data_science_projects\Recommendation_sytem/clean_data.csv')

In [3]:
data.head(3)

Unnamed: 0     budget                                             genres  \
0           0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1           1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2           2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

       id                                           keywords  \
0   19995  [{"id": 1463, "name": "culture clash"}, {"id":...   
1     285  [{"id": 270, "name": "ocean"}, {"id": 726, "na...   
2  206647  [{"id": 470, "name": "spy"}, {"id": 818, "name...   

  original_language                            original_title  \
0                en                                    Avatar   
1                en  Pirates of the Caribbean: At World's End   
2                en                                   Spectre   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   

                                production_companies release_date     revenue  \
0  [{"name": "Ingenious Film Partners", "id": 289...   2009-12-10  2787965087   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   2007-05-19   961000000   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...   2015-10-26   880674609   

   runtime                                   spoken_languages  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...   
1    169.0           [{"iso_639_1": "en", "name": "English"}]   
2    148.0  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...   

                                          tagline  vote_average  vote_count  \
0                     Enter the World of Pandora.           7.2       11800   
1  At the end of the world, the adventure begins.           6.9        4500   
2                           A Plan No One Escapes           6.3        4466   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...

In [4]:
data.shape

(4803, 19)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  spoken_languages      4803 non-null   object 
 14  tagline               3959 non-null   object 
 15  vote_average         

In [6]:
data.dtypes

Unnamed: 0                int64
budget                    int64
genres                   object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
tagline                  object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
dtype: object

### Content Based Recommendation System
    Now lets make a recommendations based on the movie’s plot summaries given in the overview column. So if our user gives us a movie title, our goal is to recommend movies that share similar plot summaries.

In [7]:
data['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [ ]:
# Considering overview column, As i have to create a recommendation engine , so for each and every movie I have to create vector of matrix
# overview col is a String or we can say its a text data and our ML model cant understand these text data,so we have to use
# NLP algo like (TF-IDF , BOW , Word-2-Vec etc..) to convert this Text data into some numerical-format or vector so that our ML Model can understand


In [8]:
data['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [9]:
data['overview'].isnull().sum()

3

In [10]:
# Filling NaNs with empty string
data['overview']=data['overview'].fillna('')

In [11]:
data['overview'].isnull().sum()

0

In [12]:
### import TF-IDF vectorizer to convert text data into numerical data
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
#ngram_range=(1, 3),-means take the diff combinations of 1 to 3 different kinds of words
#stop_words = 'english'-remove all un-necessary words like the,,that,of,she,he,is

tfv=TfidfVectorizer(min_df=3,max_features=None,ngram_range=(1,3),stop_words='english')

In [14]:
# Fitting the TF-IDF on the 'overview' text
tfv_matrix=tfv.fit_transform(data['overview'])

In [15]:
tfv_matrix

<4803x9919 sparse matrix of type '<class 'numpy.float64'>'
	with 121480 stored elements in Compressed Sparse Row format>

In [16]:
tfv_matrix.shape
#why dimension is (4803, 9919),bcz of combinatio of diff diff features just because of ngram(1,3)

## We see that over 9919 different words were used to describe the 4803 movies in our dataset.

(4803, 9919)

In [17]:
data['overview'].shape

(4803,)

In [18]:
data['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [19]:
## getting TF-IDF matrix of first entry of overview..
tfv_matrix.toarray()[0]

### Return a array representation

array([0., 0., 0., ..., 0., 0., 0.])

In [20]:
### lets transpose matrix 
tfv_matrix[0].T.toarray()

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

###  getting TF-IDF matrix of entire data

In [21]:

X=tfv_matrix.toarray()

In [22]:
df2=pd.DataFrame(X)

In [23]:
df2

0     1     2     3     4     5     6     7     8     9     ...  9909  \
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4798   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4799   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4800   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4801   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4802   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      9910  9911  9912  9913  9914  9915  9916  9917  9918  
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
...    ...   ...   ...   ...   ...   ...   ...   ...   ...  
4798   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4799   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4800   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4801   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4802   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[4803 rows x 9919 columns]

In [24]:
data['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [ ]:
#after this ,our vector is ready ,so now we have to find similarity value

In [25]:
## lets apply Sigmoid kernel
## Note-->> The function sigmoid_kernel computes the sigmoid kernel between two vectors
### https://scikit-learn.org/stable/modules/metrics.html


from sklearn.metrics.pairwise import sigmoid_kernel

In [26]:
# Compute the sigmoid kernel
sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [27]:
sig

array([[0.76163649, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76163649, 0.76159416, ..., 0.76159519, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76163649, ..., 0.76159484, 0.76159416,
        0.76159416],
       ...,
       [0.76159416, 0.76159519, 0.76159484, ..., 0.76163649, 0.76159488,
        0.76159447],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159488, 0.76163649,
        0.76159467],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159447, 0.76159467,
        0.76163649]])

In [29]:
sig[1]

array([0.76159416, 0.76163649, 0.76159416, ..., 0.76159519, 0.76159416,
       0.76159416])

In [31]:
index=data.index

In [32]:
index

RangeIndex(start=0, stop=4803, step=1)

In [33]:
index[4802]

4802

In [34]:
title=data['original_title']

In [35]:
title[4802]

'My Date with Drew'

In [36]:
data.index

RangeIndex(start=0, stop=4803, step=1)

In [38]:
# create a series of indices and movie titles
indices=pd.Series(data.index,index=data['original_title'])

In [39]:
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [40]:
indices['Avatar']

0

In [41]:
sig[0]

array([0.76163649, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [43]:
sig[indices['Avatar']]

array([0.76163649, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [44]:
list(sig[indices['Avatar']])

[0.7616364930962501,
 0.7615941559557649,
 0.7615941559557649,
 0.761595125164868,
 0.7615941559557649,
 0.761595309447695,
 0.7615941559557649,
 0.761595908255829,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615950846887262,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.76159572705,
 0.7615962614863476,
 0.7615941559557649,
 0.7615952950243241,
 0.7615941559557649,
 0.7615968611459101,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.761597988130195,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615951605621958,
 0.7615941559557649,
 0.7615968356260668,
 0.7615941559557649,
 0.7615956092492897,
 

In [45]:
len(list(sig[indices['Avatar']]))

4803

In [49]:
list(enumerate(list(sig[indices['Avatar']])))

[(0, 0.7616364930962501),
 (1, 0.7615941559557649),
 (2, 0.7615941559557649),
 (3, 0.761595125164868),
 (4, 0.7615941559557649),
 (5, 0.761595309447695),
 (6, 0.7615941559557649),
 (7, 0.761595908255829),
 (8, 0.7615941559557649),
 (9, 0.7615941559557649),
 (10, 0.7615941559557649),
 (11, 0.7615950846887262),
 (12, 0.7615941559557649),
 (13, 0.7615941559557649),
 (14, 0.7615941559557649),
 (15, 0.7615941559557649),
 (16, 0.7615941559557649),
 (17, 0.7615941559557649),
 (18, 0.7615941559557649),
 (19, 0.7615941559557649),
 (20, 0.7615941559557649),
 (21, 0.7615941559557649),
 (22, 0.7615941559557649),
 (23, 0.7615941559557649),
 (24, 0.7615941559557649),
 (25, 0.7615941559557649),
 (26, 0.76159572705),
 (27, 0.7615962614863476),
 (28, 0.7615941559557649),
 (29, 0.7615952950243241),
 (30, 0.7615941559557649),
 (31, 0.7615968611459101),
 (32, 0.7615941559557649),
 (33, 0.7615941559557649),
 (34, 0.7615941559557649),
 (35, 0.7615941559557649),
 (36, 0.761597988130195),
 (37, 0.761594155955

In [52]:
#sort similarity values in ascending order
sigma_scores=sorted(list(enumerate(list(sig[indices['Avatar']]))),key=lambda x:x[1],reverse=True)

In [53]:
sigma_scores

[(0, 0.7616364930962501),
 (1341, 0.7616030155858681),
 (634, 0.7616028561141562),
 (3604, 0.761601930611584),
 (2130, 0.7616015339622925),
 (775, 0.7616011086528327),
 (529, 0.7615996114069044),
 (151, 0.7615991171152051),
 (311, 0.7615990624497703),
 (847, 0.7615987706430225),
 (570, 0.7615986450599548),
 (942, 0.7615984376900236),
 (36, 0.761597988130195),
 (1610, 0.7615979793934843),
 (3070, 0.7615978406764746),
 (1033, 0.7615978182403835),
 (2628, 0.7615977834088159),
 (1784, 0.7615977150705628),
 (2578, 0.7615976778191441),
 (150, 0.7615976453752453),
 (3724, 0.7615975951237102),
 (1013, 0.761597590729192),
 (4211, 0.7615975631290406),
 (1213, 0.7615975380289366),
 (1345, 0.7615974549075267),
 (312, 0.7615974086679764),
 (4039, 0.7615973645677722),
 (2967, 0.7615973512232982),
 (614, 0.7615972949789032),
 (281, 0.7615972537743877),
 (174, 0.7615972462403858),
 (3493, 0.7615971922075142),
 (3624, 0.7615971821325882),
 (972, 0.7615971791001622),
 (1274, 0.7615971587672579),
 (1959,

In [54]:
sigma_scores[1:11]

[(1341, 0.7616030155858681),
 (634, 0.7616028561141562),
 (3604, 0.761601930611584),
 (2130, 0.7616015339622925),
 (775, 0.7616011086528327),
 (529, 0.7615996114069044),
 (151, 0.7615991171152051),
 (311, 0.7615990624497703),
 (847, 0.7615987706430225),
 (570, 0.7615986450599548)]

In [56]:
## to access index of above list
ind=[index[0] for index in sigma_scores[1:11]]

In [57]:
ind

[1341, 634, 3604, 2130, 775, 529, 151, 311, 847, 570]

In [58]:
data['original_title'].iloc[ind]

1341                Obitaemyy Ostrov
634                       The Matrix
3604                       Apollo 18
2130                    The American
775                        Supernova
529                 Tears of the Sun
151                          Beowulf
311     The Adventures of Pluto Nash
847                         Semi-Pro
570                           Ransom
Name: original_title, dtype: object

In [60]:
#we have to define such a function that will give recommendations


def give_rec(title,model):
    
    idx=indices[title]
    
    model_scores=list(enumerate(list(model[indices[idx]])))
    
    model_scores=sorted(model_scores,key=lambda x:x[1],reverse=True)
    
    model_scores=model_scores[1:11]
    
    movie_indices=[index[0] for index in model_scores]
    
    return data['original_title'].iloc[movie_indices]

In [61]:
# Testing our content-based recommendation system with the film 'Avatar'
give_rec('Avatar',sig)

1341                Obitaemyy Ostrov
634                       The Matrix
3604                       Apollo 18
2130                    The American
775                        Supernova
529                 Tears of the Sun
151                          Beowulf
311     The Adventures of Pluto Nash
847                         Semi-Pro
570                           Ransom
Name: original_title, dtype: object

## Improve Existing Content Based Recommendation System..

In [ ]:
### Consider more features like Credits, Genres and Keywords that can impact my recommendations..

In [62]:
data.columns

Index(['Unnamed: 0', 'budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [64]:
data['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [ ]:
### now u have to access director name if job belongs to director , but in above case  , data sounds extremely pretty bad..
### hence it is good to provide some structure , hence u can think of using eval or literal_eval ..

In [65]:
eval(data['crew'][0])

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [66]:
from ast import literal_eval

In [67]:
literal_eval(data['crew'][0])

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [68]:
# Parse the stringified features into their corresponding python objects

features=['cast','crew','keywords','genres']

In [69]:
for feature in features:
    data[feature]=data[feature].apply(literal_eval)

In [70]:
data['crew'][1]

[{'credit_id': '52fe4232c3a36847f800b579',
  'department': 'Camera',
  'gender': 2,
  'id': 120,
  'job': 'Director of Photography',
  'name': 'Dariusz Wolski'},
 {'credit_id': '52fe4232c3a36847f800b4fd',
  'department': 'Directing',
  'gender': 2,
  'id': 1704,
  'job': 'Director',
  'name': 'Gore Verbinski'},
 {'credit_id': '52fe4232c3a36847f800b54f',
  'department': 'Production',
  'gender': 2,
  'id': 770,
  'job': 'Producer',
  'name': 'Jerry Bruckheimer'},
 {'credit_id': '52fe4232c3a36847f800b503',
  'department': 'Writing',
  'gender': 2,
  'id': 1705,
  'job': 'Screenplay',
  'name': 'Ted Elliott'},
 {'credit_id': '52fe4232c3a36847f800b509',
  'department': 'Writing',
  'gender': 2,
  'id': 1706,
  'job': 'Screenplay',
  'name': 'Terry Rossio'},
 {'credit_id': '52fe4232c3a36847f800b57f',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '52fe4232c3a36847f800b585',
  'department': 'Editing',
  'gender': 2,
 

In [71]:
data[features]

cast  \
0     [{'cast_id': 242, 'character': 'Jake Sully', '...   
1     [{'cast_id': 4, 'character': 'Captain Jack Spa...   
2     [{'cast_id': 1, 'character': 'James Bond', 'cr...   
3     [{'cast_id': 2, 'character': 'Bruce Wayne / Ba...   
4     [{'cast_id': 5, 'character': 'John Carter', 'c...   
...                                                 ...   
4798  [{'cast_id': 1, 'character': 'El Mariachi', 'c...   
4799  [{'cast_id': 1, 'character': 'Buzzy', 'credit_...   
4800  [{'cast_id': 8, 'character': 'Oliver O’Toole',...   
4801  [{'cast_id': 3, 'character': 'Sam', 'credit_id...   
4802  [{'cast_id': 3, 'character': 'Herself', 'credi...   

                                                   crew  \
0     [{'credit_id': '52fe48009251416c750aca23', 'de...   
1     [{'credit_id': '52fe4232c3a36847f800b579', 'de...   
2     [{'credit_id': '54805967c3a36829b5002c41', 'de...   
3     [{'credit_id': '52fe4781c3a36847f81398c3', 'de...   
4     [{'credit_id': '52fe479ac3a36847f813eaa3', 'de...   
...                                                 ...   
4798  [{'credit_id': '52fe44eec3a36847f80b280b', 'de...   
4799  [{'credit_id': '52fe487dc3a368484e0fb013', 'de...   
4800  [{'credit_id': '52fe4df3c3a36847f8275ecf', 'de...   
4801  [{'credit_id': '52fe4ad9c3a368484e16a36b', 'de...   
4802  [{'credit_id': '58ce021b9251415a390165d9', 'de...   

                                               keywords  \
0     [{'id': 1463, 'name': 'culture clash'}, {'id':...   
1     [{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...   
2     [{'id': 470, 'name': 'spy'}, {'id': 818, 'name...   
3     [{'id': 849, 'name': 'dc comics'}, {'id': 853,...   
4     [{'id': 818, 'name': 'based on novel'}, {'id':...   
...                                                 ...   
4798  [{'id': 5616, 'name': 'united states–mexico ba...   
4799                                                 []   
4800  [{'id': 248, 'name': 'date'}, {'id': 699, 'nam...   
4801                                                 []   
4802  [{'id': 1523, 'name': 'obsession'}, {'id': 224...   

                                                 genres  
0     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  
1     [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  
2     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  
3     [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...  
4     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  
...                                                 ...  
4798  [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...  
4799  [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...  
4800  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  
4801                                                 []  
4802                [{'id': 99, 'name': 'Documentary'}]  

[4803 rows x 4 columns]

In [72]:
data['crew'][0]

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [73]:
'shan '.strip()

'shan'

In [74]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name'].strip()
        
    else:
        return np.nan

In [75]:
get_director(data['crew'][0])

'James Cameron'

In [76]:
data['director']=data['crew'].apply(get_director)

In [78]:
data['director'].shape

(4803,)

In [79]:
data['director'].isnull().sum()

30

In [80]:
data['cast'][0]

[{'cast_id': 242,
  'character': 'Jake Sully',
  'credit_id': '5602a8a7c3a3685532001c9a',
  'gender': 2,
  'id': 65731,
  'name': 'Sam Worthington',
  'order': 0},
 {'cast_id': 3,
  'character': 'Neytiri',
  'credit_id': '52fe48009251416c750ac9cb',
  'gender': 1,
  'id': 8691,
  'name': 'Zoe Saldana',
  'order': 1},
 {'cast_id': 25,
  'character': 'Dr. Grace Augustine',
  'credit_id': '52fe48009251416c750aca39',
  'gender': 1,
  'id': 10205,
  'name': 'Sigourney Weaver',
  'order': 2},
 {'cast_id': 4,
  'character': 'Col. Quaritch',
  'credit_id': '52fe48009251416c750ac9cf',
  'gender': 2,
  'id': 32747,
  'name': 'Stephen Lang',
  'order': 3},
 {'cast_id': 5,
  'character': 'Trudy Chacon',
  'credit_id': '52fe48009251416c750ac9d3',
  'gender': 1,
  'id': 17647,
  'name': 'Michelle Rodriguez',
  'order': 4},
 {'cast_id': 8,
  'character': 'Selfridge',
  'credit_id': '52fe48009251416c750ac9e1',
  'gender': 2,
  'id': 1771,
  'name': 'Giovanni Ribisi',
  'order': 5},
 {'cast_id': 7,
  'c

In [81]:
type(data['cast'][0])

list

In [83]:
'Shan singh '.strip().lower()

'shan singh'

In [85]:
# Returns the list top 3 elements or entire list , whichever is more..
def get_list(x):
    names=[i['name'].strip().lower() for i in x if type(x)==list]
    
    if len(names)>3:
        return names[0:3]
    else:
        return names

In [ ]:
'''
def get_list(x):
    names=[]
    for i in x:
        if type(x)==list:
            name=i['name']
            name=name.strip()
            name=name.lower()
            names.append(name)
    
    if len(names)>3:
        return names[0:3]
    else:
        return names
        
'''

In [86]:
get_list(data['crew'][0])

['stephen e. rivkin', 'rick carter', 'christopher boyes']

In [87]:
df2=data.copy()

In [88]:
for feature in features:
    data[feature]=data[feature].apply(get_list)

In [89]:
data[features]

cast  \
0      [sam worthington, zoe saldana, sigourney weaver]   
1         [johnny depp, orlando bloom, keira knightley]   
2          [daniel craig, christoph waltz, léa seydoux]   
3          [christian bale, michael caine, gary oldman]   
4        [taylor kitsch, lynn collins, samantha morton]   
...                                                 ...   
4798  [carlos gallardo, jaime de hoyos, peter marqua...   
4799       [edward burns, kerry bishé, marsha dietlein]   
4800         [eric mabius, kristin booth, crystal lowe]   
4801          [daniel henney, eliza coupe, bill paxton]   
4802  [drew barrymore, brian herzlinger, corey feldman]   

                                                   crew  \
0     [stephen e. rivkin, rick carter, christopher b...   
1     [dariusz wolski, gore verbinski, jerry bruckhe...   
2             [thomas newman, sam mendes, anna pinnock]   
3       [hans zimmer, charles roven, christopher nolan]   
4       [andrew stanton, andrew stanton, john lasseter]   
...                                                 ...   
4798  [robert rodriguez, robert rodriguez, robert ro...   
4799         [edward burns, edward burns, edward burns]   
4800       [carla hetland, harvey kahn, adam sliwinski]   
4801                         [daniel hsia, daniel hsia]   
4802  [clark peterson, andrew reimer, brian herzlinger]   

                                        keywords  \
0             [culture clash, future, space war]   
1             [ocean, drug abuse, exotic island]   
2            [spy, based on novel, secret agent]   
3          [dc comics, crime fighter, terrorist]   
4              [based on novel, mars, medallion]   
...                                          ...   
4798  [united states–mexico barrier, legs, arms]   
4799                                          []   
4800      [date, love at first sight, narration]   
4801                                          []   
4802               [obsession, camcorder, crush]   

                                    genres  
0             [action, adventure, fantasy]  
1             [adventure, fantasy, action]  
2               [action, adventure, crime]  
3                   [action, crime, drama]  
4     [action, adventure, science fiction]  
...                                    ...  
4798             [action, crime, thriller]  
4799                     [comedy, romance]  
4800              [comedy, drama, romance]  
4801                                    []  
4802                         [documentary]  

[4803 rows x 4 columns]

In [90]:
df2[features]

cast  \
0     [{'cast_id': 242, 'character': 'Jake Sully', '...   
1     [{'cast_id': 4, 'character': 'Captain Jack Spa...   
2     [{'cast_id': 1, 'character': 'James Bond', 'cr...   
3     [{'cast_id': 2, 'character': 'Bruce Wayne / Ba...   
4     [{'cast_id': 5, 'character': 'John Carter', 'c...   
...                                                 ...   
4798  [{'cast_id': 1, 'character': 'El Mariachi', 'c...   
4799  [{'cast_id': 1, 'character': 'Buzzy', 'credit_...   
4800  [{'cast_id': 8, 'character': 'Oliver O’Toole',...   
4801  [{'cast_id': 3, 'character': 'Sam', 'credit_id...   
4802  [{'cast_id': 3, 'character': 'Herself', 'credi...   

                                                   crew  \
0     [{'credit_id': '52fe48009251416c750aca23', 'de...   
1     [{'credit_id': '52fe4232c3a36847f800b579', 'de...   
2     [{'credit_id': '54805967c3a36829b5002c41', 'de...   
3     [{'credit_id': '52fe4781c3a36847f81398c3', 'de...   
4     [{'credit_id': '52fe479ac3a36847f813eaa3', 'de...   
...                                                 ...   
4798  [{'credit_id': '52fe44eec3a36847f80b280b', 'de...   
4799  [{'credit_id': '52fe487dc3a368484e0fb013', 'de...   
4800  [{'credit_id': '52fe4df3c3a36847f8275ecf', 'de...   
4801  [{'credit_id': '52fe4ad9c3a368484e16a36b', 'de...   
4802  [{'credit_id': '58ce021b9251415a390165d9', 'de...   

                                               keywords  \
0     [{'id': 1463, 'name': 'culture clash'}, {'id':...   
1     [{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...   
2     [{'id': 470, 'name': 'spy'}, {'id': 818, 'name...   
3     [{'id': 849, 'name': 'dc comics'}, {'id': 853,...   
4     [{'id': 818, 'name': 'based on novel'}, {'id':...   
...                                                 ...   
4798  [{'id': 5616, 'name': 'united states–mexico ba...   
4799                                                 []   
4800  [{'id': 248, 'name': 'date'}, {'id': 699, 'nam...   
4801                                                 []   
4802  [{'id': 1523, 'name': 'obsession'}, {'id': 224...   

                                                 genres  
0     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  
1     [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  
2     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  
3     [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...  
4     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  
...                                                 ...  
4798  [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...  
4799  [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...  
4800  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  
4801                                                 []  
4802                [{'id': 99, 'name': 'Documentary'}]  

[4803 rows x 4 columns]

In [91]:
data.isnull().sum()

Unnamed: 0                0
budget                    0
genres                    0
id                        0
keywords                  0
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
release_date              1
revenue                   0
runtime                   2
spoken_languages          0
tagline                 844
vote_average              0
vote_count                0
cast                      0
crew                      0
director                 30
dtype: int64

In [93]:
data.dropna(subset=['director'],inplace=True)

In [94]:
data.isnull().sum()

Unnamed: 0                0
budget                    0
genres                    0
id                        0
keywords                  0
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
release_date              0
revenue                   0
runtime                   2
spoken_languages          0
tagline                 822
vote_average              0
vote_count                0
cast                      0
crew                      0
director                  0
dtype: int64

In [ ]:
'''
We are now in a position to create our "metadata " as important_feature which is a string 
that contains all the metadata that we want to feed to our vectorizer 'Countvectorizer' (namely actors, director and keywords)

'''

In [95]:
data['cast'][0]

['sam worthington', 'zoe saldana', 'sigourney weaver']

In [98]:
type(data['cast'][0])

list

In [96]:
' '.join(data['cast'][0])

'sam worthington zoe saldana sigourney weaver'

In [97]:
type(' '.join(data['cast'][0]))

str

In [99]:
data.columns

Index(['Unnamed: 0', 'budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [100]:
data[features]

cast  \
0      [sam worthington, zoe saldana, sigourney weaver]   
1         [johnny depp, orlando bloom, keira knightley]   
2          [daniel craig, christoph waltz, léa seydoux]   
3          [christian bale, michael caine, gary oldman]   
4        [taylor kitsch, lynn collins, samantha morton]   
...                                                 ...   
4798  [carlos gallardo, jaime de hoyos, peter marqua...   
4799       [edward burns, kerry bishé, marsha dietlein]   
4800         [eric mabius, kristin booth, crystal lowe]   
4801          [daniel henney, eliza coupe, bill paxton]   
4802  [drew barrymore, brian herzlinger, corey feldman]   

                                                   crew  \
0     [stephen e. rivkin, rick carter, christopher b...   
1     [dariusz wolski, gore verbinski, jerry bruckhe...   
2             [thomas newman, sam mendes, anna pinnock]   
3       [hans zimmer, charles roven, christopher nolan]   
4       [andrew stanton, andrew stanton, john lasseter]   
...                                                 ...   
4798  [robert rodriguez, robert rodriguez, robert ro...   
4799         [edward burns, edward burns, edward burns]   
4800       [carla hetland, harvey kahn, adam sliwinski]   
4801                         [daniel hsia, daniel hsia]   
4802  [clark peterson, andrew reimer, brian herzlinger]   

                                        keywords  \
0             [culture clash, future, space war]   
1             [ocean, drug abuse, exotic island]   
2            [spy, based on novel, secret agent]   
3          [dc comics, crime fighter, terrorist]   
4              [based on novel, mars, medallion]   
...                                          ...   
4798  [united states–mexico barrier, legs, arms]   
4799                                          []   
4800      [date, love at first sight, narration]   
4801                                          []   
4802               [obsession, camcorder, crush]   

                                    genres  
0             [action, adventure, fantasy]  
1             [adventure, fantasy, action]  
2               [action, adventure, crime]  
3                   [action, crime, drama]  
4     [action, adventure, science fiction]  
...                                    ...  
4798             [action, crime, thriller]  
4799                     [comedy, romance]  
4800              [comedy, drama, romance]  
4801                                    []  
4802                         [documentary]  

[4773 rows x 4 columns]

In [101]:
def create_feature(row):
    return ' '.join(row['keywords']) + ' ' + ' '.join(row['cast']) + ' ' + row['director'] + ' ' + ' '.join(row['genres'])

In [102]:
data['important_feature']=data.apply(create_feature,axis=1)

In [103]:
data.head(2)

Unnamed: 0     budget                        genres     id  \
0           0  237000000  [action, adventure, fantasy]  19995   
1           1  300000000  [adventure, fantasy, action]    285   

                             keywords original_language  \
0  [culture clash, future, space war]                en   
1  [ocean, drug abuse, exotic island]                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   

                                production_companies  ...     revenue  \
0  [{"name": "Ingenious Film Partners", "id": 289...  ...  2787965087   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...  ...   961000000   

   runtime                                   spoken_languages  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...   
1    169.0           [{"iso_639_1": "en", "name": "English"}]   

                                          tagline vote_average  vote_count  \
0                     Enter the World of Pandora.          7.2       11800   
1  At the end of the world, the adventure begins.          6.9        4500   

                                               cast  \
0  [sam worthington, zoe saldana, sigourney weaver]   
1     [johnny depp, orlando bloom, keira knightley]   

                                                crew        director  \
0  [stephen e. rivkin, rick carter, christopher b...   James Cameron   
1  [dariusz wolski, gore verbinski, jerry bruckhe...  Gore Verbinski   

                                   important_feature  
0  culture clash future space war sam worthington...  
1  ocean drug abuse exotic island johnny depp orl...  

[2 rows x 21 columns]

In [104]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

In [105]:
count=CountVectorizer(stop_words='english')

In [106]:
count_matrix=count.fit_transform(data['important_feature'])

In [107]:
count_matrix

<4773x10750 sparse matrix of type '<class 'numpy.int64'>'
	with 66282 stored elements in Compressed Sparse Row format>

In [108]:
count_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [109]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [110]:
cosine_sim2=cosine_similarity(count_matrix,count_matrix)

In [111]:
cosine_sim2

array([[1.        , 0.1875    , 0.1875    , ..., 0.        , 0.        ,
        0.        ],
       [0.1875    , 1.        , 0.125     , ..., 0.        , 0.        ,
        0.        ],
       [0.1875    , 0.125     , 1.        , ..., 0.        , 0.16666667,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.16666667, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [112]:
cosine_sim2[0]

array([1.    , 0.1875, 0.1875, ..., 0.    , 0.    , 0.    ])

In [114]:
data['original_title'][6]

'Tangled'

In [115]:
give_rec('Tangled',cosine_sim2)

578     Alvin and the Chipmunks: The Squeakquel
1108                                  Pinocchio
1481                         The House of Magic
1857                            Rugrats Go Wild
42                                  Toy Story 3
390                          Hotel Transylvania
565                                     Shrek 2
899                                       Shrek
1695                                    Aladdin
3281                        The Secret of Kells
Name: original_title, dtype: object

In [116]:
give_rec('The Godfather',cosine_sim2)

867     The Godfather: Part III
2731     The Godfather: Part II
1525             Apocalypse Now
2792        Glengarry Glen Ross
1209              The Rainmaker
3012              The Outsiders
4209           The Conversation
2649          The Son of No One
1018            The Cotton Club
4124         This Thing of Ours
Name: original_title, dtype: object

In [117]:
give_rec('The Godfather',sig)

2731                             The Godfather: Part II
1873                                         Blood Ties
3730                                              Cargo
867                             The Godfather: Part III
3623                                               Made
3963                                 They Came Together
2464                             The Master of Disguise
2973    For Greater Glory - The True Story of Cristiada
2389                                        Renaissance
1475                                  Playing for Keeps
Name: original_title, dtype: object

In [ ]:
'''

We see that our recommender has been successful in capturing more information due to more metadata
and has given us better recommendations.


In order to improve more , We can also increase the weight of the director,by adding the feature multiple times in the soup.

'''